In [32]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras import regularizers
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [3]:
data = pd.read_csv('/Users/inigoparra/Desktop/carpeta sin título/clean-master.csv')

In [4]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

X_train = train_data['text']
y_train = train_data['label']

X_test = test_data['text']
y_test = test_data['label']

X_val = val_data['text']
y_val = val_data['label']

In [5]:
with open('/Users/inigoparra/Desktop/GitHub Repositories/WiBaSets/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
        
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    return ' '.join(filtered_tokens)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)
X_val = X_val.apply(preprocess_text)

In [6]:
import re
import nltk

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[0-9]', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and emojis
    text = text.split()  # Tokenize the text
    text = ' '.join(text)
    return text

X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)
X_val = X_val.apply(clean_text)

print(X_train[1])

irailaren eaj bozkatzea izango herri gisa gizarte gisa aurrera egitea


In [7]:
max_features = 20500
max_len = 100
embedding_dim = 300

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)

In [36]:
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=max_len))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(GRU(64, return_sequences=True,
               kernel_regularizer=regularizers.l2(0.02), 
               recurrent_regularizer=regularizers.l2(0.02))))
model.add(Dropout(0.7))
model.add(GRU(32))
model.add(Dropout(0.7))
model.add(Dense(3, activation='softmax'))

optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
batch_size = 16
epochs = 15
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/15
439/439 [==============================] - 20s 40ms/step - loss: 2.6958 - accuracy: 0.3666 - val_loss: 2.2222 - val_accuracy: 0.3513
Epoch 2/15
439/439 [==============================] - 17s 40ms/step - loss: 1.9017 - accuracy: 0.3808 - val_loss: 1.6558 - val_accuracy: 0.3577
Epoch 3/15
439/439 [==============================] - 17s 40ms/step - loss: 1.4799 - accuracy: 0.3962 - val_loss: 1.3628 - val_accuracy: 0.3359
Epoch 4/15
439/439 [==============================] - 18s 40ms/step - loss: 1.2472 - accuracy: 0.4322 - val_loss: 1.2133 - val_accuracy: 0.3718
Epoch 5/15
439/439 [==============================] - 18s 40ms/step - loss: 1.0231 - accuracy: 0.5711 - val_loss: 1.1954 - val_accuracy: 0.4090
Epoch 6/15
439/439 [==============================] - 18s 40ms/step - loss: 0.7967 - accuracy: 0.6723 - val_loss: 1.2943 - val_accuracy: 0.4295
Epoch 7/15
439/439 [==============================] - 18s 40ms/step - loss: 0.6221 - accuracy: 0.7784 - val_loss: 1.4621 - val_accuracy:

In [31]:
y_predict = model.predict(X_val_pad)
y_predict_classes = np.argmax(y_predict, axis=1)
print(classification_report(y_val, y_predict_classes, digits=3))

25/25 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

           0      0.268     0.111     0.157       234
           1      0.368     0.845     0.513       284
           2      0.226     0.027     0.048       262

    accuracy                          0.350       780
   macro avg      0.287     0.328     0.239       780
weighted avg      0.290     0.350     0.250       780



In [20]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

28/28 [==============================] - 0s 9ms/step - loss: 2.0049 - accuracy: 0.3679
Test loss: 2.00490665435791, Test accuracy: 0.3679354190826416
